In [22]:
import os
import numpy as np
import pandas as pd
import gc # garbage collector

# 시각화 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# 머신러닝 라이브러리
# 1	StandardScaler:	기본 스케일(가장 많이 사용). 평균과 표준편차 사용 -> 표준 정규분포의 형태로
# 2	MinMaxScaler:	최대/최소값이 각각 1, 0이 되도록 스케일링
# 3	MaxAbsScaler:	최대절대값과 0이 각각 1, 0이 되도록 스케일링
# 4	RobustScaler:	중앙값(median)과 IQR(interquartile range) 사용. 아웃라이어의 영향을 최소화
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from IPython.display import display


import warnings
warnings.filterwarnings('ignore')

In [12]:
# os.listdir('./drive/MyDrive/colaboratory/machine_learning_data')

['train.csv', 'test.csv']

In [2]:
# train, test data 불러오기
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [3]:
# data 사이즈 확인
print('train_df.shape : ',train_df.shape)

# data 사이즈 확인
print('test_df.shape : ',test_df.shape)

train_df.shape :  (4459, 4993)
test_df.shape :  (49342, 4992)


In [4]:
train_df.head(3)

,ID,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000d6aaf2,38000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,000fbd867,600000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,0027d6b71,10000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [5]:
test_df.head(3)

,ID,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000137c73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00021489f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0004d7953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Train, Test Data 정보를 확인하기

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Columns: 4993 entries, ID to 9fc776466
dtypes: float64(1845), int64(3147), object(1)
memory usage: 169.9+ MB


In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49342 entries, 0 to 49341
Columns: 4992 entries, ID to 9fc776466
dtypes: float64(4991), object(1)
memory usage: 1.8+ GB


In [8]:
# 데이터 전처리(preprocessing)
# 결측치 유무
train_df.isnull().sum().sum()

0

In [9]:
# 데이터 전처리(preprocessing)
# 결측치 유무
test_df.isnull().sum().sum()

0

In [10]:
# 모델 학습에 의미없는 상수 컬럼 삭제

colsToRemove = []
for col in train_df.columns:
  if col != 'ID' and col != 'target':
    if train_df[col].std() == 0 :
      colsToRemove.append(col)

train_df.drop(colsToRemove, axis =1 ,inplace = True)
test_df.drop(colsToRemove, axis =1 ,inplace = True)

In [11]:
print('삭제된 컬럼 수 : ', len(colsToRemove))
print('삭제된 컬럼: ', colsToRemove)

삭제된 컬럼 수 :  256
삭제된 컬럼:  ['d5308d8bc', 'c330f1a67', 'eeac16933', '7df8788e8', '5b91580ee', '6f29fbbc7', '46dafc868', 'ae41a98b6', 'f416800e9', '6d07828ca', '7ac332a1d', '70ee7950a', '833b35a7c', '2f9969eab', '8b1372217', '68322788b', '2288ac1a6', 'dc7f76962', '467044c26', '39ebfbfd9', '9a5ff8c23', 'f6fac27c8', '664e2800e', 'ae28689a2', 'd87dcac58', '4065efbb6', 'f944d9d43', 'c2c4491d5', 'a4346e2e2', '1af366d4f', 'cfff5b7c8', 'da215e99e', '5acd26139', '9be9c6cef', '1210d0271', '21b0a54cb', 'da35e792b', '754c502dd', '0b346adbd', '0f196b049', 'b603ed95d', '2a50e001c', '1e81432e7', '10350ea43', '3c7c7e24c', '7585fce2a', '64d036163', 'f25d9935c', 'd98484125', '95c85e227', '9a5273600', '746cdb817', '6377a6293', '7d944fb0c', '87eb21c50', '5ea313a8c', '0987a65a1', '2fb7c2443', 'f5dde409b', '1ae50d4c3', '2b21cd7d8', '0db8a9272', '804d8b55b', '76f135fa6', '7d7182143', 'f88e61ae6', '378ed28e0', 'ca4ba131e', '1352ddae5', '2b601ad67', '6e42ff7c7', '22196a84c', '0e410eb3d', '992e6d1d3', '90a742107',

In [12]:
# 중복 컬럼 삭제(중복 row 아님)
groups = train_df.columns.to_series().groupby(train_df.dtypes).groups

dup_col = []

for key,value in groups.items():
  df_col = train_df[value].columns
  df = train_df[value]
  l_df_col = len(df_col)

  for i in range(l_df_col):
    i_df = df.iloc[:,i].values
    for j in range(i+1,l_df_col):
      j_df = df.iloc[:,j].values
      if np.array_equal(i_df,j_df):
        dup_col.append(df_col[i])
        break

print(dup_col)

train_df = train_df.drop(dup_col, axis =1)
test_df = test_df.drop(dup_col, axis =1)

['34ceb0081', '8d57e2749', '168b3e5bc', 'a765da8bc', 'acc5b709d']


In [13]:
# 희소한 데이터를 삭제 - 데이터가 1~2 개 밖에 없는것, 개수가 너무 unique 한 경우를 의미한다.
temp_col = [x for x in train_df.columns if not x in ['ID','target']]

for temp in temp_col:
  if len(np.unique(train_df[temp]))<2:
    train_df.drop(temp,axis=1,inplace=True)
    test_df.drop(temp,axis=1,inplace=True)

In [14]:
# 불필요한 메모리 소모 요소 제거
gc.collect()

print("Train set size: {}".format(train_df.shape))

print("Test set size: {}".format(test_df.shape))

Train set size: (4459, 4732)
Test set size: (49342, 4731)


In [15]:
# Prepare Data for Training & Test
x_train = train_df.drop(["ID", "target"], axis=1)

# np.log1p는 np.log와 다르다.
# x에 0값이 들어갈 수 있기 떄문에 -> 기본적으로 1을 보정값으로 줘서 1이 될 수 없게 만든다. np.log1p(t) => t = x+1
y_train = np.log1p(train_df["target"].values)

x_test = test_df.drop(["ID"], axis=1)

# train_test_split, 학습용 데이터와 검증용 데이터를 나눴다. radom_state - 똑같은 결과를 추출하기 위해서 설정해준다.
dev_x, val_x, dev_y, val_y = train_test_split(x_train, y_train, test_size = 0.2, random_state = 2021)

In [16]:
# Build LightGBM Model & Predict -> 예측, 해석력도 어느 정도 좋다.
params = {
    'objective' : 'regression',
    'metric' : 'rmse', # root mse 사용
    'num_leaves' : 40, # tree 기반이기 때문에 DT 에서 사용한 것 사용
    'learning_rate' : 0.004, # 학습률
    'bagging_fraction': 0.6, # 앙상블 - 샘플을 뽑는 비율
    'feature_fraction': 0.6, # feature 다양성을 위해서 feature 60% 에서 뽑아 다양하게 만들겠다.
    'bagging_frequnecy' : 6,
    'bagging_seed' : 2021,
    'verbosity': -1, # 학습을 한세트씩 계속 보여주겠다.
    'seed' : 2021
}

# LGB 를 위한 Dataset 을 만들어주기
lgtrain = lgb.Dataset(dev_x, dev_y)
lgval = lgb.Dataset(val_x, val_y)

# 학습된 모델들의 결과들을 계속 넣어줄 것이다.
evals_result = {}

# model 학습 실시
num_boost_round = 5000
model = lgb.train(params=params,
                  train_set=lgtrain,
                  num_boost_round=num_boost_round,
                  valid_sets=[lgtrain, lgval])  # 로그 출력 주기 설정

In [17]:
# np.log1p()를 다시 되돌리는 exp => np.expm1(), 최적의 모델로 학습하겠다는 의미
pred_test_y = np.expm1(model.predict(x_test,num_iteration=model.best_iteration))
print("LightGBM Training Completed...")

# np.log1p()를 다시 되돌리는 exp => np.expm1()
pred_test_y = np.expm1(model.predict(x_test,num_iteration=model.best_iteration))

LightGBM Training Completed...


In [18]:
# feature importance
print("Features Importance...")
gain = model.feature_importance('gain')
featureimp = pd.DataFrame({
    'feature':model.feature_name(),
    'split':model.feature_importance('split'),
    'gain':100*gain/gain.sum() # 비율로써 보여준다.
}).sort_values('gain',ascending=False) # 내림차순 정렬을 통해 잘한 순으로 보여준다.

print(featureimp[:50])

Features Importance...
        feature  split      gain
4130  f190486d6   1821  7.621016
2375  58e2e02e6   2173  4.618078
4020  15ace8c9f   1493  2.802241
3465  eeb9cd3aa   1485  2.500979
2614  9fd594eec   1453  2.048108
3661  491b9ee45    769  1.710021
3571  58232a6fb    988  1.223688
1457  b43a7cfd5    960  1.118673
834   6eef030c1   1052  0.981092
3722  d6bb78916   1066  0.972435
8     20aa07010   1009  0.951043
3220  ced6a7e91    515  0.865832
3867  2288333b4    395  0.784080
917   e222309b0    277  0.745603
1482  024c577b9    794  0.702911
2687  fb0f5dbfe   1182  0.696830
4343  1702b5bf0    700  0.640280
4185  f74e8f13d    909  0.611349
2211  1931ccfdd    549  0.593348
4508  c47340d97    640  0.568075
863   fc99f9426    508  0.526184
3779  70feb1494    743  0.506864
1548  26ab20ff9    238  0.503751
828   6786ea46d    323  0.500063
1378  6cf7866c1    201  0.491232
1007  1c71183bb    580  0.471722
537   26fc93eb7    641  0.469548
1783  c9eda7d9c    730  0.462680
1748  5f341a818    4

In [19]:
# Build XGBoost Model & Predict
params = {
    'objective' : 'reg:linear',
    'eval_metric' : 'rmse',
    'eta' : 0.001,
    'max_depth' : 10,
    'subsample' : 0.6,
    'colsample_bytree' : 0.6,
    'alpha' : 0.001,
    'random_state' : 2021,
    'silent' : True
}

tr_data = xgb.DMatrix(dev_x,dev_y)
va_data = xgb.DMatrix(val_x,val_y)

watchlist = [(tr_data,'train'), (va_data,'valid')]

model_xgb = xgb.train(
    params = params,
    dtrain = tr_data,
    num_boost_round = 2000,
    evals = watchlist,
    maximize = False,
    early_stopping_rounds = 100,
    verbose_eval = 100
)

[0]	train-rmse:1.75591	valid-rmse:1.72725
[100]	train-rmse:1.70075	valid-rmse:1.69300
[200]	train-rmse:1.65044	valid-rmse:1.66254
[300]	train-rmse:1.60368	valid-rmse:1.63492
[400]	train-rmse:1.56074	valid-rmse:1.61045
[500]	train-rmse:1.52098	valid-rmse:1.58887
[600]	train-rmse:1.48391	valid-rmse:1.56948
[700]	train-rmse:1.44931	valid-rmse:1.55271
[800]	train-rmse:1.41745	valid-rmse:1.53770
[900]	train-rmse:1.38769	valid-rmse:1.52415
[1000]	train-rmse:1.36061	valid-rmse:1.51196
[1100]	train-rmse:1.33515	valid-rmse:1.50165
[1200]	train-rmse:1.31102	valid-rmse:1.49236
[1300]	train-rmse:1.28861	valid-rmse:1.48426
[1400]	train-rmse:1.26726	valid-rmse:1.47684
[1500]	train-rmse:1.24676	valid-rmse:1.47007
[1600]	train-rmse:1.22874	valid-rmse:1.46421
[1700]	train-rmse:1.21084	valid-rmse:1.45870
[1800]	train-rmse:1.19399	valid-rmse:1.45389
[1900]	train-rmse:1.17825	valid-rmse:1.44982
[1999]	train-rmse:1.16357	valid-rmse:1.44581


In [28]:
dtest = xgb.DMatrix(x_test)
xgb_pred_y = np.expm1(model_xgb.predict(dtest, iteration_range=(0, model_xgb.best_iteration)))
print("XGBoost Training Completed...")


XGBoost Training Completed...


In [29]:
cb_model = CatBoostRegressor(
    iterations = 500,
    learning_rate = 0.05,
    depth = 10,
    eval_metric = 'RMSE',
    random_seed = 2021,
    bagging_temperature = 0.2,
    od_type = 'Iter',
    metric_period = 50,
    od_wait = 20
)

cb_model.fit(
    dev_x,dev_y,
    eval_set = (val_x,val_y),
    use_best_model = True,
    verbose = 50
)

0:	learn: 1.7489421	test: 1.7216299	best: 1.7216299 (0)	total: 2.37s	remaining: 19m 40s
50:	learn: 1.4888660	test: 1.5406275	best: 1.5406275 (50)	total: 1m 46s	remaining: 15m 40s
100:	learn: 1.3800745	test: 1.4993093	best: 1.4993093 (100)	total: 3m 41s	remaining: 14m 34s
150:	learn: 1.3099000	test: 1.4801917	best: 1.4800142 (149)	total: 5m 37s	remaining: 13m
200:	learn: 1.2439396	test: 1.4686800	best: 1.4686800 (200)	total: 7m 31s	remaining: 11m 11s
250:	learn: 1.1701638	test: 1.4587038	best: 1.4585832 (249)	total: 9m 24s	remaining: 9m 19s
300:	learn: 1.1084993	test: 1.4524398	best: 1.4524156 (299)	total: 11m 21s	remaining: 7m 30s
350:	learn: 1.0686589	test: 1.4473458	best: 1.4473458 (350)	total: 13m 20s	remaining: 5m 39s
400:	learn: 1.0293033	test: 1.4444994	best: 1.4441743 (398)	total: 15m 11s	remaining: 3m 45s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 1.444174333
bestIteration = 398

Shrink model to first 399 iterations.


In [30]:
# Compare Loss between LightGB vs XGBoost vs Catboost
# Combine Predictions
total_predict = pd.DataFrame()

total_predict["lgb_pred"] = pred_test_y

total_predict["xgb_pred"] = xgb_pred_y

total_predict["cat_pred"] = pred_test_cat

# 각 모델의 loss sum을 해보면, 각 모델의 성능을 비교해볼 수 있다.
# 하지만.. 모델의 파라미터 최적화는 따로 시키지 않았기 때문에 지금의 모델 비교는 약간 의미가 없긴 하다.
# "이렇게 모델을 비교할 수도 있구나" 정도로만 봐주면 좋겠다.
total_predict

NameError: name 'pred_test_cat' is not defined